# Pre Split Test Analysis
### Can we detect difference in KPI in one business cycle?

In [1]:
%load_ext rpy2.ipython

## What we know

In [ ]:
from datetime import datetime, timedelta

alpha = 0.05
beta = 0.80
business_cycle_length = timedelta(days=7)
base_n = 220


In [6]:
%R power.prop.test(p1=0.1, p2=0.11, power=0.8, alternative='two.sided', sig.level=0.05)

R object with classes: ('power.htest',) mapped to:
[Flo..., Flo..., Flo..., Flo..., Flo..., Str..., Str..., Str...]
  n: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7f32e83d2e40> [RTYPES.REALSXP]
  p1: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7f32e9593480> [RTYPES.REALSXP]
  p2: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7f32e6b0e180> [RTYPES.REALSXP]
  sig.level: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7f32e6b0e480> [RTYPES.REALSXP]
  power: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7f32e6b0e4c0> [RTYPES.REALSXP]
  alternative: <class 'rpy2.rinterface_lib.sexp.StrSexpVector'>
  <rpy2.rinterface_lib.sexp.StrSexpVector object at 0x7f32e6b0e680> [RTYPES.STRSXP]
  note: <class 'rpy2.rinterface_lib.sexp.StrSexpVector'>
  <rpy2.rinterface_lib.sexp.StrSexpVector object at 0x7f32e6b0e980> [RTYPES.STRSXP]
  method: <class 'rpy2.rinterface_lib.sexp.StrSexpVector'>
  <rpy2.rinterface_lib.sexp.StrSexpVector object at 0x7f32e6b0e9c0> [RTYPES.STRSXP]